<a href="https://colab.research.google.com/github/kavyaa2015/MepcoTraining/blob/main/assignment_LLM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

1) Pretraining  

            a. Using distilgpt for textgeneration

            b. Chat generation using mistralai/Mistral-7B-Instruct-v0.2

            c. Summarize text using T5-small

In [ ]:
#a. Using distilgpt for textgeneration
from transformers import pipeline
generator = pipeline("text-generation", model="distilgpt2")
result = generator(
    "Once in a while, engineers need to recalibrate their instruments to maintain accuracy.",
    max_new_tokens=50,
    temperature=0.7,
    top_p=0.9,
    top_k=50,
    repetition_penalty=1.2
)
print(result[0]['generated_text'])


Device set to use cuda:0
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Once in a while, engineers need to recalibrate their instruments to maintain accuracy.
‹ The problem is that the instrument-making process of using this technology takes more than one year and can take longer if you are not ready for it—and they often fail because an error occurs at multiple stages (for example, when some


In [ ]:
# b. Chat generation using mistralai/Mistral-7B-Instruct-v0.2
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

model = "mistralai/Mistral-7B-Instruct-v0.2"
generator = pipeline("text-generation", model=model, torch_dtype="auto", device_map="auto")

prompt = "User: Explain what causes dopamine release.\nAssistant:"
response = generator(prompt, max_new_tokens=100, temperature=0.7)
print(response[0]['generated_text'])

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
`torch_dtype` is deprecated! Use `dtype` instead!


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Device set to use cuda:0
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


User: Explain what causes dopamine release.
Assistant: Dopamine is a type of neurotransmitter, a chemical messenger that transmits signals in the brain. It's often associated with feelings of pleasure and reward, and plays a key role in motivation, motivation-reward learning, and in regulating movement and emotion.

Dopamine is released in response to various stimuli. Here are some common causes:

1. Reward: Experiencing something pleasurable or desirable can trigger the release of dop


In [ ]:
#c. Summarize text using T5-small
from transformers import pipeline
summarizer = pipeline("summarization", model="t5-small")

text = """
Climate change is one of the biggest challenges facing humanity today. Rising global temperatures,
melting ice caps, and extreme weather events are disrupting ecosystems and communities worldwide.
Governments and organizations are working together to reduce carbon emissions, promote renewable
energy, and develop sustainable technologies. However, significant efforts are still needed to slow
down global warming and protect future generations from its devastating impacts.
"""

summary = summarizer(text, max_length=50, min_length=25, do_sample=False)
print(summary[0]['summary_text'])



Device set to use cuda:0
Both `max_new_tokens` (=256) and `max_length`(=50) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


climate change is one of the biggest challenges facing humanity today . rising global temperatures, melting ice caps, and extreme weather events disrupt ecosystems and communities .


In [ ]:
import kagglehub
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import AutoTokenizer, AutoModelForSequenceClassification, AdamW

# LOAD KAGGLE CSV
path = kagglehub.dataset_download("lakshmi25npathi/imdb-dataset-of-50k-movie-reviews")
csv_path = path + "/IMDB Dataset.csv"

df = pd.read_csv(csv_path)
df["label"] = df["sentiment"].map({"positive": 1, "negative": 0})
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

class IMDBDataset(Dataset):
    def __init__(self, df):
        self.texts = df["review"].tolist()
        self.labels = df["label"].tolist()

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        enc = tokenizer(
            self.texts[idx],
            truncation=True,
            padding="max_length",
            max_length=256,
            return_tensors="pt"
        )
        item = {k: v.squeeze(0) for k, v in enc.items()}
        item["labels"] = torch.tensor(self.labels[idx])
        return item

train_df = df.sample(frac=0.8, random_state=42)
test_df  = df.drop(train_df.index)

train_loader = DataLoader(IMDBDataset(train_df), batch_size=8, shuffle=True)
test_loader  = DataLoader(IMDBDataset(test_df), batch_size=8)

# LOAD DISTILBERT

device = "cuda" if torch.cuda.is_available() else "cpu"

model = AutoModelForSequenceClassification.from_pretrained(
    "distilbert-base-uncased",
    num_labels=2
).to(device)

optimizer = AdamW(model.parameters(), lr=3e-5)

#  TRAIN
model.train()
for epoch in range(1):
    for step, batch in enumerate(train_loader):
        batch = {k: v.to(device) for k, v in batch.items()}
        out = model(**batch)

        loss = out.loss
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if step % 200 == 0:
            print(f"Step {step} | Loss {loss.item():.4f}")

print("Training finished.")


Using Colab cache for faster access to the 'imdb-dataset-of-50k-movie-reviews' dataset.


Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.12/dist-packages/transformers/optimization.py:588: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Step 0 | Loss 0.6684
Step 200 | Loss 0.3644
Step 400 | Loss 0.0719
Step 600 | Loss 0.4582
Step 800 | Loss 0.5697
Step 1000 | Loss 0.0705
Step 1200 | Loss 0.3928
Step 1400 | Loss 0.0471
Step 1600 | Loss 0.4585
Step 1800 | Loss 0.2278
Step 2000 | Loss 0.3031
Step 2200 | Loss 0.4241
Step 2400 | Loss 0.0132
Step 2600 | Loss 0.0949
Step 2800 | Loss 0.3028
Step 3000 | Loss 0.1030
Step 3200 | Loss 0.3622
Step 3400 | Loss 0.1025
Step 3600 | Loss 0.0630
Step 3800 | Loss 0.0473
Step 4000 | Loss 0.4543
Step 4200 | Loss 0.0838
Step 4400 | Loss 0.1802
Step 4600 | Loss 0.3051
Step 4800 | Loss 0.0286
Training finished.


In [ ]:
# EXAMPLE SENTIMENT CHECK
model.eval()

def predict_sentiment(text):
    enc = tokenizer(
        text,
        truncation=True,
        padding="max_length",
        max_length=256,
        return_tensors="pt"
    ).to(device)

    with torch.no_grad():
        outputs = model(**enc)
        logits = outputs.logits
        predicted = torch.argmax(logits, dim=1).item()

    sentiment = "Positive" if predicted == 1 else "Negative"
    return sentiment

# Default example checks
sample1 = "This movie was absolutely wonderful. I loved every moment!"
sample2 = "This movie was not bad."

print("Sample 1:", predict_sentiment(sample1))
print("Sample 2:", predict_sentiment(sample2))


Sample 1: Positive
Sample 2: Positive


In [ ]:
!pip install transformers -q

from transformers import pipeline

# Load SQuAD fine-tuned QA model
qa = pipeline("question-answering", model="deepset/roberta-base-squad2")

print("QA Chatbot ready! Type 'exit' to quit.\n")

context = """
The Eiffel Tower, constructed in 1889 for the Paris World's Fair, is one of the
most famous landmarks in the world. Designed by engineer Gustave Eiffel, the
tower stands 324 meters tall and was originally criticized by many artists and
intellectuals. Today, it attracts millions of visitors every year and serves as
a global symbol of France and Paris.


"""

while True:
    question = input("You: ")

    if question.lower() == "exit":
        print("Bot: Goodbye!")
        break

    result = qa({"question": question, "context": context})
    print("Bot:", result["answer"])


Device set to use cpu


QA Chatbot ready! Type 'exit' to quit.

You: how do people view the tower today?
Bot: a global symbol
You: exit
Bot: Goodbye!


In [ ]:
!pip install transformers sentencepiece accelerate --quiet

from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

model_name = "google/flan-t5-base"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

def ask(prompt):
    inputs = tokenizer(prompt, return_tensors="pt")
    output = model.generate(**inputs, max_new_tokens=80)
    return tokenizer.decode(output[0], skip_special_tokens=True)


# Contexts & Questions
context_1 = """
The Nile River is the longest river in Africa. It flows through 11 countries
and is a major source of water for agriculture and transportation.
"""

question_1 = "How many countries does the Nile River flow through?"

context_2 = """
COVID-19 is a respiratory disease caused by the SARS-CoV-2 virus.
It first appeared in late 2019 and led to a global pandemic. Vaccines
were introduced in 2021 to reduce severe infections.
"""

question_2 = "When were COVID-19 vaccines introduced?"

# 1. Zero-shot prompting
print("ZERO-SHOT PROMPT")
prompt = f"""
Answer the question based only on the context.

Context: {context_1}
Question: {question_1}
"""
answer = ask(prompt)
print("Question:", question_1)
print("Answer:", answer)
# 2. One-shot prompting
print("ONE-SHOT PROMPT")
example = """
Context: The Sun is a star at the center of the solar system.
Question: What is the Sun?
Answer: A star at the center of the solar system.
"""

prompt = f"""
Use the example to answer the new question.

Example:
{example}

Now answer:
Context: {context_2}
Question: {question_2}
"""
answer = ask(prompt)
print("Question:", question_2)
print("Answer:", answer)

# 3. Few-shot prompting
print("FEW-SHOT PROMPT")
examples = """
Context: Water boils at 100°C at sea level.
Question: When does water boil?
Answer: At 100°C.

Context: The Great Wall of China is over 13,000 miles long.
Question: How long is the Great Wall of China?
Answer: Over 13,000 miles.
"""

prompt = f"""
Answer the question using the examples as reference.

Examples:
{examples}

Now answer:
Context: {context_1}
Question: {question_1}
"""
answer = ask(prompt)
print("Question:", question_1)
print("Answer:", answer)

# 4. Chain-of-thought prompting
print("CHAIN-OF-THOUGHT PROMPT")
prompt = f"""
Think step-by-step and then answer.

Context: {context_2}
Question: {question_2}
"""
answer = ask(prompt)
print("Question:", question_2)
print("Answer:", answer)

# 5. Role prompting
print("ROLE PROMPT")
prompt = f"""
You are a historian. Provide a clear answer.

Context: {context_1}
Question: {question_1}
"""
answer = ask(prompt)
print("Question:", question_1)
print("Answer:", answer)

# 6. Extractive QA prompting
print("EXTRACTIVE QA PROMPT")
prompt = f"""
Extract the answer exactly from the context. Use only the words found in the text.

Context: {context_2}
Question: {question_2}
"""
answer = ask(prompt)
print("Question:", question_2)
print("Answer:", answer)

ZERO-SHOT PROMPT
Question: How many countries does the Nile River flow through?
Answer: 11
ONE-SHOT PROMPT
Question: When were COVID-19 vaccines introduced?
Answer: 2021
FEW-SHOT PROMPT
Question: How many countries does the Nile River flow through?
Answer: 11
CHAIN-OF-THOUGHT PROMPT
Question: When were COVID-19 vaccines introduced?
Answer: 2021. So the final answer is 2021.
ROLE PROMPT
Question: How many countries does the Nile River flow through?
Answer: 11
EXTRACTIVE QA PROMPT
Question: When were COVID-19 vaccines introduced?
Answer: 2021


In [ ]:
# Install required libraries
!pip install transformers datasets sentencepiece sumy nltk --quiet

# Download required NLTK models
import nltk
nltk.download("punkt")
nltk.download("punkt_tab")

from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from sumy.parsers.plaintext import PlaintextParser
from sumy.nlp.tokenizers import Tokenizer
from sumy.summarizers.text_rank import TextRankSummarizer

# Input text
text = """
Artificial intelligence (AI) has rapidly transformed multiple industries.
From healthcare and finance to transportation and education, AI systems are
being used to assist professionals, automate repetitive tasks, and offer
data-driven insights. AI models such as neural networks learn patterns
from large datasets, allowing them to perform tasks such as image recognition,
language translation, and decision-making. Despite the advantages, concerns
exist regarding job displacement, privacy issues, and bias in AI systems.
Researchers and policymakers are working to ensure that AI development
remains safe, transparent, and beneficial to society.
"""

# ----------------------------
# 1. Extractive Summarization (TextRank)
# ----------------------------
def textrank_summary(text, sentence_count=3):
    parser = PlaintextParser.from_string(text, Tokenizer("english"))
    summarizer = TextRankSummarizer()
    summary_sentences = summarizer(parser.document, sentence_count)
    return " ".join(str(sentence) for sentence in summary_sentences)

extractive = textrank_summary(text)


# ----------------------------
# 2. Abstractive Summarization (Hugging Face FLAN-T5)
# ----------------------------
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

model_name = "facebook/bart-large-cnn"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

prompt = "Summarize the following text in 3-4 sentences: " + text
inputs = tokenizer(prompt, return_tensors="pt", truncation=True)

summary_ids = model.generate(
    inputs.input_ids,
    num_beams=4,
    max_new_tokens=150,
    min_length=50
)

abstractive = tokenizer.decode(summary_ids[0], skip_special_tokens=True)


# ----------------------------
# Comparison Output
# ----------------------------
print("Original Text Length:", len(text.split()), "words\n")

print("EXTRACTIVE SUMMARY (TextRank):")
print(extractive)
print("\nLength:", len(extractive.split()), "words\n")

print("ABSTRACTIVE SUMMARY (FLAN-T5):")
print(abstractive)
print("\nLength:", len(abstractive.split()), "words\n")


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Original Text Length: 87 words

EXTRACTIVE SUMMARY (TextRank):
From healthcare and finance to transportation and education, AI systems are being used to assist professionals, automate repetitive tasks, and offer data-driven insights. AI models such as neural networks learn patterns from large datasets, allowing them to perform tasks such as image recognition, language translation, and decision-making. Researchers and policymakers are working to ensure that AI development remains safe, transparent, and beneficial to society.

Length: 64 words

ABSTRACTIVE SUMMARY (FLAN-T5):
Artificial intelligence (AI) has rapidly transformed multiple industries. Despite the advantages, concerns exist regarding job displacement, privacy issues, and bias in AI systems. Researchers and policymakers are working to ensure that AI development is safe, transparent, and beneficial to society.

Length: 40 words



In [ ]:
from transformers import pipeline

# Retrieval-based
retrieval_classifier = pipeline("zero-shot-classification", model="facebook/bart-large-mnli")
responses = {
    "technology": "New technologies like AI and blockchain are changing the world rapidly.",
    "greeting": "Hello! Nice to meet you. How can I help you today?",
    "wellbeing": "I'm just a bot, but I'm feeling great! How about you?"
}
candidate_labels = list(responses.keys())

def retrieval_response(user_input):
    result = retrieval_classifier(user_input, candidate_labels)
    best_label = result['labels'][0]
    return responses[best_label]

# Generative (GPT-2)
generative_chatbot = pipeline("text-generation", model="gpt2")

def generative_response(user_input):
    prompt = f"User: {user_input}\nChatbot:"
    response = generative_chatbot(prompt, max_length=50, do_sample=True, temperature=0.6)
    text = response[0]['generated_text'][len(prompt):].strip()
    # Take only the first line to avoid rambling
    return text.split("\n")[0]

# Single example input
example_input = "Can you tell me something about technology?"

retrieval_reply = retrieval_response(example_input)
generative_reply = generative_response(example_input)

print("===================================")
print(f"User: {example_input}\n")
print("Retrieval-Based Chatbot:", retrieval_reply)
print("Generative Chatbot   :", generative_reply)
print("===================================")


Device set to use cpu
Device set to use cpu
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Both `max_new_tokens` (=256) and `max_length`(=50) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


User: Can you tell me something about technology?

Retrieval-Based Chatbot: New technologies like AI and blockchain are changing the world rapidly.
Generative Chatbot   : It's not a new technology; it was introduced in the mid-1990s. It's not a new technology. It was introduced in the mid-1990s. It's not a new technology.
